In [1]:
import torch
import scipy.sparse as sp
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
from torch.optim import Adam
from sklearn.preprocessing import LabelEncoder

# Load the processed data
combined_sparse = sp.load_npz('data/combined_sparse.npz')
df_targets = pd.read_csv('data/df_targets.csv')
df_targets['PRIM_CONTRIBUTORY_CAUSE'] = pd.Categorical(df_targets['PRIM_CONTRIBUTORY_CAUSE'])

# Encode the target variable
label_encoder = LabelEncoder()
df_targets_encoded = pd.DataFrame()
df_targets_encoded['PRIM_CONTRIBUTORY_CAUSE_LABEL'] = label_encoder.fit_transform(df_targets['PRIM_CONTRIBUTORY_CAUSE'].cat.codes)

# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(
    combined_sparse,
    df_targets_encoded['PRIM_CONTRIBUTORY_CAUSE_LABEL'].values,
    test_size=0.2,
    random_state=42
)

# Convert the target arrays to PyTorch tensors
y_train_tensor = torch.tensor(y_train).long()
y_test_tensor = torch.tensor(y_test).long()

# Set the device to use for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Move the data to the device
x_train = torch.sparse_coo_tensor(x_train.nonzero(), x_train.data, x_train.shape, device=device).float()
x_test = torch.sparse_coo_tensor(x_test.nonzero(), x_test.data, x_test.shape, device=device).float()
y_train_tensor = y_train_tensor.to(device)
y_test_tensor = y_test_tensor.to(device)

# Create the training dataset and data loader
train_dataset = TensorDataset(x_train, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Define the neural network model
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Set the hyperparameters
input_size = combined_sparse.shape[1]
hidden_size = 128
num_classes = len(label_encoder.classes_)
learning_rate = 0.001
num_epochs = 20

# Create an instance of the neural network model
model = NeuralNetwork(input_size, hidden_size, num_classes)

# Move the model to the device
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for batch_data, batch_targets in train_loader:
        # Move the batch data and targets to the device
        batch_data = batch_data.to(device)
        batch_targets = batch_targets.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        logits = model(batch_data)

        # Calculate the loss
        loss = criterion(logits, batch_targets)

        # Backward pass and optimization step
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
with torch.no_grad():
    # Forward pass
    logits = model(x_test)

    # Get the predicted labels
    _, predicted = torch.max(logits.data, 1)

    # Calculate the accuracy
    accuracy = (predicted == y_test_tensor).sum().item() / y_test_tensor.size(0)

print("Accuracy:", accuracy)


C:\Users\gerba\AppData\Local\Temp\ipykernel_37128\1130341577.py:36: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:264.)
  x_train = torch.sparse_coo_tensor(x_train.nonzero(), x_train.data, x_train.shape, device=device).float()
c:\Anaconda3\lib\site-packages\torch\optim\optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.is_available():


Accuracy: 0.7930218805440568


In [2]:
import matplotlib.pyplot as plt

# Choose a feature index to analyze (e.g., 0 for the first feature)
feature_index = 0

# Create a copy of the test dataset
x_test_copy = x_test.clone()

# Define the range of values for the chosen feature
feature_values = torch.linspace(torch.min(x_test_copy[:, feature_index]), torch.max(x_test_copy[:, feature_index]), steps=100)

# Set the chosen feature to the specified values while keeping the other features unchanged
x_test_copy[:, feature_index] = feature_values

# Forward pass with the modified dataset
logits = model(x_test_copy)

# Calculate the average predicted probability for each feature value
average_predictions = torch.mean(torch.softmax(logits, dim=1), dim=0)

# Plot the partial dependence plot
plt.plot(feature_values.cpu().numpy(), average_predictions.cpu().numpy())
plt.xlabel('Feature Value')
plt.ylabel('Average Predicted Probability')
plt.title('Partial Dependence Plot')
plt.show()


NotImplementedError: Could not run 'aten::as_strided' with arguments from the 'SparseCUDA' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::as_strided' is only available for these backends: [CPU, CUDA, Meta, QuantizedCPU, QuantizedCUDA, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradHIP, AutogradXLA, AutogradMPS, AutogradIPU, AutogradXPU, AutogradHPU, AutogradVE, AutogradLazy, AutogradMeta, AutogradMTIA, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, AutogradNestedTensor, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorc

In [ ]:
import shap

# Create an explainer object
explainer = shap.Explainer(model, x_train)

# Compute SHAP values for the test dataset
shap_values = explainer.shap_values(x_test)

# Summarize the SHAP values
shap.summary_plot(shap_values, feature_names=feature_names, plot_type='bar')


c:\Anaconda3\lib\site-packages\shap\utils\_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
c:\Anaconda3\lib\site-packages\shap\utils\_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def delta_minimization_order(all_masks, max_swap_size=100, num_passes=2):
c:\Anac

AttributeError: module 'warnings' has no attribute 'DeprecationWarning'

In [ ]:
from lime import lime_tabular

# Create an explainer object
explainer = lime_tabular.LimeTabularExplainer(x_train, feature_names=feature_names)

# Choose a specific instance for explanation (e.g., the first instance in the test set)
instance_index = 0
instance = x_test[instance_index].cpu().numpy()

# Generate explanations for the chosen instance
explanation = explainer.explain_instance(instance, model.predict_proba, num_features=len(feature_names))

# Show the explanation
explanation.show_in_notebook()


NameError: name 'feature_names' is not defined

# Older Models

In [ ]:
# import torch
# import scipy.sparse as sp
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from torch.utils.data import DataLoader, TensorDataset
# from torch import nn
# from torch.optim import Adam
# from sklearn.preprocessing import LabelEncoder

# # Load the processed data
# combined_sparse = sp.load_npz('data/combined_sparse.npz')
# df_targets = pd.read_csv('data/df_targets.csv')
# df_targets['PRIM_CONTRIBUTORY_CAUSE'] = pd.Categorical(df_targets['PRIM_CONTRIBUTORY_CAUSE'])

# # Encode the target variable
# label_encoder = LabelEncoder()
# df_targets_encoded = pd.DataFrame()
# df_targets_encoded['PRIM_CONTRIBUTORY_CAUSE_LABEL'] = label_encoder.fit_transform(df_targets['PRIM_CONTRIBUTORY_CAUSE'].cat.codes)

# # Split the data into train and test sets
# x_train, x_test, y_train, y_test = train_test_split(
#     combined_sparse,
#     df_targets_encoded['PRIM_CONTRIBUTORY_CAUSE_LABEL'].values,
#     test_size=0.2,
#     random_state=42
# )

# # Convert the target arrays to PyTorch tensors
# y_train_tensor = torch.tensor(y_train).long()
# y_test_tensor = torch.tensor(y_test).long()

# # Set the device to use for training
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(device)
# # Move the data to the device
# x_train = torch.sparse_coo_tensor(x_train.nonzero(), x_train.data, x_train.shape, device=device).float()
# x_test = torch.sparse_coo_tensor(x_test.nonzero(), x_test.data, x_test.shape, device=device).float()
# y_train_tensor = y_train_tensor.to(device)
# y_test_tensor = y_test_tensor.to(device)

# # Create the training dataset and data loader
# train_dataset = TensorDataset(x_train, y_train_tensor)
# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# # Define the neural network model
# class NeuralNetwork(nn.Module):
#     def __init__(self, input_size, hidden_size, num_classes):
#         super(NeuralNetwork, self).__init__()
#         self.fc1 = nn.Linear(input_size, hidden_size)
#         self.relu = nn.ReLU()
#         self.fc2 = nn.Linear(hidden_size, num_classes)

#     def forward(self, x):
#         out = self.fc1(x)
#         out = self.relu(out)
#         out = self.fc2(out)
#         return out

# # Set the hyperparameters
# input_size = combined_sparse.shape[1]
# hidden_size = 64
# num_classes = len(label_encoder.classes_)
# learning_rate = 0.001
# num_epochs = 10

# # Create an instance of the neural network model
# model = NeuralNetwork(input_size, hidden_size, num_classes)

# # Move the model to the device
# model = model.to(device)

# # Define the loss function and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = Adam(model.parameters(), lr=learning_rate)

# # Training loop
# for epoch in range(num_epochs):
#     model.train()
#     for batch_data, batch_targets in train_loader:
#         # Move the batch data and targets to the device
#         batch_data = batch_data.to(device)
#         batch_targets = batch_targets.to(device)

#         # Zero the gradients
#         optimizer.zero_grad()

#         # Forward pass
#         logits = model(batch_data)

#         # Calculate the loss
#         loss = criterion(logits, batch_targets)

#         # Backward pass and optimization step
#         loss.backward()
#         optimizer.step()

# # Evaluation
# model.eval()
# with torch.no_grad():
#     # Forward pass
#     logits = model(x_test)

#     # Get the predicted labels
#     _, predicted = torch.max(logits.data, 1)

#     # Calculate the accuracy
#     accuracy = (predicted == y_test_tensor).sum().item() / y_test_tensor.size(0)

# print("Accuracy:", accuracy)


cuda


C:\Users\gerba\AppData\Local\Temp\ipykernel_31816\2837531019.py:36: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:264.)
  x_train = torch.sparse_coo_tensor(x_train.nonzero(), x_train.data, x_train.shape, device=device).float()
c:\Anaconda3\lib\site-packages\torch\optim\optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.is_available():


Accuracy: 0.7953625931621909


In [ ]:
# import torch
# import scipy.sparse as sp
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from torch.utils.data import DataLoader, TensorDataset
# from torch import nn
# from torch.optim import Adam
# from sklearn.preprocessing import LabelEncoder

# # Load the processed data
# combined_sparse = sp.load_npz('data/combined_sparse.npz')
# df_targets = pd.read_csv('data/df_targets.csv')
# df_targets['PRIM_CONTRIBUTORY_CAUSE'] = pd.Categorical(df_targets['PRIM_CONTRIBUTORY_CAUSE'])

# # Encode the target variable
# label_encoder = LabelEncoder()
# df_targets_encoded = pd.DataFrame()
# df_targets_encoded['PRIM_CONTRIBUTORY_CAUSE_LABEL'] = label_encoder.fit_transform(df_targets['PRIM_CONTRIBUTORY_CAUSE'].cat.codes)

# # Split the data into train and test sets
# x_train, x_test, y_train, y_test = train_test_split(
#     combined_sparse,
#     df_targets_encoded['PRIM_CONTRIBUTORY_CAUSE_LABEL'].values,
#     test_size=0.2,
#     random_state=42
# )

# # Convert the target arrays to PyTorch tensors
# y_train_tensor = torch.tensor(y_train).long()
# y_test_tensor = torch.tensor(y_test).long()

# # Set the device to use for training
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Move the data to the device
# x_train = torch.sparse_coo_tensor(x_train.nonzero(), x_train.data, x_train.shape, device=device).float()
# x_test = torch.sparse_coo_tensor(x_test.nonzero(), x_test.data, x_test.shape, device=device).float()
# y_train_tensor = y_train_tensor.to(device)
# y_test_tensor = y_test_tensor.to(device)

# # Create the training dataset and data loader
# train_dataset = TensorDataset(x_train, y_train_tensor)
# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# # Define the neural network model
# class NeuralNetwork(nn.Module):
#     def __init__(self, input_size, hidden_size, num_classes):
#         super(NeuralNetwork, self).__init__()
#         self.fc1 = nn.Linear(input_size, hidden_size)
#         self.relu = nn.ReLU()
#         self.fc2 = nn.Linear(hidden_size, num_classes)

#     def forward(self, x):
#         out = self.fc1(x)
#         out = self.relu(out)
#         out = self.fc2(out)
#         return out

# # Set the hyperparameters
# input_size = combined_sparse.shape[1]
# hidden_size = 128
# num_classes = len(label_encoder.classes_)
# learning_rate = 0.001
# num_epochs = 10

# # Create an instance of the neural network model
# model = NeuralNetwork(input_size, hidden_size, num_classes)

# # Move the model to the device
# model = model.to(device)

# # Define the loss function and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = Adam(model.parameters(), lr=learning_rate)

# # Training loop
# for epoch in range(num_epochs):
#     model.train()
#     for batch_data, batch_targets in train_loader:
#         # Move the batch data and targets to the device
#         batch_data = batch_data.to(device)
#         batch_targets = batch_targets.to(device)

#         # Zero the gradients
#         optimizer.zero_grad()

#         # Forward pass
#         logits = model(batch_data)

#         # Calculate the loss
#         loss = criterion(logits, batch_targets)

#         # Backward pass and optimization step
#         loss.backward()
#         optimizer.step()

# # Evaluation
# model.eval()
# with torch.no_grad():
#     # Forward pass
#     logits = model(x_test)

#     # Get the predicted labels
#     _, predicted = torch.max(logits.data, 1)

#     # Calculate the accuracy
#     accuracy = (predicted == y_test_tensor).sum().item() / y_test_tensor.size(0)

# print("Accuracy:", accuracy)


c:\Anaconda3\lib\site-packages\torch\optim\optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.is_available():


Accuracy: 0.7701644386608305


In [ ]:
# import torch
# import scipy.sparse as sp
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from torch.utils.data import DataLoader, TensorDataset
# from torch import nn
# from torch.optim import Adam
# from sklearn.preprocessing import LabelEncoder

# # Load the processed data
# combined_sparse = sp.load_npz('data/combined_sparse.npz')
# df_targets = pd.read_csv('data/df_targets.csv')
# df_targets['PRIM_CONTRIBUTORY_CAUSE'] = pd.Categorical(df_targets['PRIM_CONTRIBUTORY_CAUSE'])

# # Encode the target variable
# label_encoder = LabelEncoder()
# df_targets_encoded = pd.DataFrame()
# df_targets_encoded['PRIM_CONTRIBUTORY_CAUSE_LABEL'] = label_encoder.fit_transform(df_targets['PRIM_CONTRIBUTORY_CAUSE'].cat.codes)

# # Split the data into train and test sets
# x_train, x_test, y_train, y_test = train_test_split(
#     combined_sparse,
#     df_targets_encoded['PRIM_CONTRIBUTORY_CAUSE_LABEL'].values,
#     test_size=0.2,
#     random_state=42
# )

# # Convert the target arrays to PyTorch tensors
# y_train_tensor = torch.tensor(y_train).long()
# y_test_tensor = torch.tensor(y_test).long()

# # Set the device to use for training
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(device)
# # Move the data to the device
# x_train = torch.sparse_coo_tensor(x_train.nonzero(), x_train.data, x_train.shape, device=device).float()
# x_test = torch.sparse_coo_tensor(x_test.nonzero(), x_test.data, x_test.shape, device=device).float()
# y_train_tensor = y_train_tensor.to(device)
# y_test_tensor = y_test_tensor.to(device)

# # Create the training dataset and data loader
# train_dataset = TensorDataset(x_train, y_train_tensor)
# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# # Define the neural network model
# class NeuralNetwork(nn.Module):
#     def __init__(self, input_size, hidden_size, num_classes):
#         super(NeuralNetwork, self).__init__()
#         self.fc1 = nn.Linear(input_size, hidden_size)
#         self.relu = nn.ReLU()
#         self.fc2 = nn.Linear(hidden_size, num_classes)

#     def forward(self, x):
#         out = self.fc1(x)
#         out = self.relu(out)
#         out = self.fc2(out)
#         return out

# # Set the hyperparameters
# input_size = combined_sparse.shape[1]
# hidden_size = 128
# num_classes = len(label_encoder.classes_)
# learning_rate = 0.001
# num_epochs = 10

# # Create an instance of the neural network model
# model = NeuralNetwork(input_size, hidden_size, num_classes)

# # Move the model to the device
# model = model.to(device)

# # Define the loss function and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = Adam(model.parameters(), lr=learning_rate)

# # Training loop
# for epoch in range(num_epochs):
#     model.train()
#     for batch_data, batch_targets in train_loader:
#         # Move the batch data and targets to the device
#         batch_data = batch_data.to(device)
#         batch_targets = batch_targets.to(device)

#         # Zero the gradients
#         optimizer.zero_grad()

#         # Forward pass
#         logits = model(batch_data)

#         # Calculate the loss
#         loss = criterion(logits, batch_targets)

#         # Backward pass and optimization step
#         loss.backward()
#         optimizer.step()

# # Evaluation
# model.eval()
# with torch.no_grad():
#     # Forward pass
#     logits = model(x_test)

#     # Get the predicted labels
#     _, predicted = torch.max(logits.data, 1)

#     # Calculate the accuracy
#     accuracy = (predicted == y_test_tensor).sum().item() / y_test_tensor.size(0)

# print("Accuracy:", accuracy)


cuda


c:\Anaconda3\lib\site-packages\torch\optim\optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.is_available():


Accuracy: 0.7796521944871643
